# Bayes Estimator

### <font color='orange'>Universidad Autónoma de Yucatán</font> _Facultad de Matemáticas_

**Teacher:** Dr. Arturo Espinosa Romero <[eromero@correo.uady.mx](mailto:eromero@correo.uady.mx)>

**Student:** Ing. Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Practice 5: Extended Kalman Filter for Ball Tracking

## GitHub: [Practice 5](https://github.com/dayan3847/bayes_estimator/tree/main/practice5-ball_tracking)

In [8]:
import sympy as sp
import numpy as np

In [9]:
#@title Estado de sistema
X, Y, Z, = sp.symbols('X Y Z')
dX, dY, dZ = sp.symbols('\dot{X} \dot{Y} \dot{Z}')
# El estado esta compuesto por las coordenadas reales y las velocidades
XX = sp.Matrix([
    [X],
    [Y],
    [Z],
    [dX],
    [dY],
    [dZ],
])

sp.Eq(sp.Symbol('\mathbb{X}'), XX, evaluate=False)

Eq(\mathbb{X}, Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [10]:
#@title Matriz A "transitionMatrix"
dt = sp.Symbol('\Delta t')
# Esta seria la matrix de transformacion de estado
AA = sp.Matrix([
    [1, 0, 0, dt, 0, 0],
    [0, 1, 0, 0, dt, 0],
    [0, 0, 1, 0, 0, dt],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
])
sp.Eq(sp.Symbol('\mathbb{A}'), AA, evaluate=False)

Eq(\mathbb{A}, Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]]))

In [11]:
#@title "Z" Medicion
x, y, r = sp.symbols('x y r')
dx, dy = sp.symbols('\dot{x} \dot{y}')
# La medicion esta compuesta por las coordenadas de la camara y el tamanno del radio (en metros)
ZZ = sp.Matrix([
    [x],
    [y],
    [r],
    [dx],
    [dy],
])
sp.Eq(sp.Symbol('\mathbb{Z}'), ZZ, evaluate=False)

Eq(\mathbb{Z}, Matrix([
[      x],
[      y],
[      r],
[\dot{x}],
[\dot{y}]]))

In [12]:
# @title Funcion h
Rm = sp.Symbol('Rm')  # Radio real de la pelota en metros
h_x = sp.Matrix([
    [X / Z],  # x
    [Y / Z],  # y
    [Rm / Z],  # r
    [-X * dZ / Z ** 2 + dX / Z],  # dx
    [-Y * dZ / Z ** 2 + dY / Z],  # dy
])
sp.Eq(sp.Function('h')(sp.Symbol('\mathbb{X}')), h_x, evaluate=False)

Eq(h(\mathbb{X}), Matrix([
[                        X/Z],
[                        Y/Z],
[                       Rm/Z],
[-X*\dot{Z}/Z**2 + \dot{X}/Z],
[-Y*\dot{Z}/Z**2 + \dot{Y}/Z]]))

In [13]:
# @title Matriz H es el Jacobian de h respecto al estado
HH = h_x.jacobian(XX)
sp.Eq(sp.Symbol('\mathbb{H}x'), HH, evaluate=False)

Eq(\mathbb{H}x, Matrix([
[          1/Z,             0,                         -X/Z**2,   0,   0,       0],
[            0,           1/Z,                         -Y/Z**2,   0,   0,       0],
[            0,             0,                        -Rm/Z**2,   0,   0,       0],
[-\dot{Z}/Z**2,             0, 2*X*\dot{Z}/Z**3 - \dot{X}/Z**2, 1/Z,   0, -X/Z**2],
[            0, -\dot{Z}/Z**2, 2*Y*\dot{Z}/Z**3 - \dot{Y}/Z**2,   0, 1/Z, -Y/Z**2]]))

In [16]:
rows, cols = HH.shape
for r in range(rows):
    line = ''
    for c in range(cols):
        line += str(
            HH[r, c]
            .subs(dX, sp.Symbol('dX'))
            .subs(dY, sp.Symbol('dY'))
            .subs(dZ, sp.Symbol('dZ'))
        ) + ', '
    print(line)

1/Z, 0, -X/Z**2, 0, 0, 0, 
0, 1/Z, -Y/Z**2, 0, 0, 0, 
0, 0, -Rm/Z**2, 0, 0, 0, 
-dZ/Z**2, 0, 2*X*dZ/Z**3 - dX/Z**2, 1/Z, 0, -X/Z**2, 
0, -dZ/Z**2, 2*Y*dZ/Z**3 - dY/Z**2, 0, 1/Z, -Y/Z**2, 
